## Connector demo
compatible with EDC v0.10.1

most examples based on the edc samples: https://github.com/eclipse-edc/Samples

In [43]:
import requests
import json

from dataspace_apis import *

### Demo setup

In [44]:

IS_LOCALHOST_DEPLOYMENT = True

# for paas:
#PROVIDER_URL = "https://ad4gd-provider-edc.dashboard-siba.store"
#CONSUMER_URL = "https://consumer-edc-connector.apps.dcw1.paas.psnc.pl"
#FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.dcw1.paas.psnc.pl"

PROVIDER_URL = "https://provider-edc-connector.apps.paas-dev.psnc.pl"
CONSUMER_URL = "https://consumer-edc-connector.apps.paas-dev.psnc.pl"
FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.paas-dev.psnc.pl"

#PROVIDER_URL = "https://provider-edc-connector.apps.dcw1.paas.psnc.pl"
#CONSUMER_URL = "https://consumer-edc-connector.apps.dcw1.paas.psnc.pl"
#FEDERATED_CATALOG_BASE_URL = "https://federatedcatalog-edc-connector.apps.dcw1.paas.psnc.pl"

## for local:
LOCALHOST = "http://localhost"
CONSUMER_CONTAINER = "http://consumer-connector"
PROVIDER_CONTAINER = "http://provider-connector"
FEDERATED_CATALOG_BASE_CONTAINER = "http://federated-catalog"

if (IS_LOCALHOST_DEPLOYMENT):
    CONSUMER_URL = LOCALHOST
    PROVIDER_URL = LOCALHOST
    FEDERATED_CATALOG_BASE_URL = LOCALHOST

In [45]:
PROVIDER_API = f"{PROVIDER_URL}/api"
PROVIDER_CONTROL = f"{PROVIDER_URL}/control"
PROVIDER_MANAGEMENT = f"{PROVIDER_URL}/management"
PROVIDER_PROTOCOL = f"{PROVIDER_URL}/protocol"
PROVIDER_PUBLIC = f"{PROVIDER_URL}/public"

CONSUMER_API = f"{CONSUMER_URL}/api"
CONSUMER_CONTROL = f"{CONSUMER_URL}/control"
CONSUMER_MANAGEMENT = f"{CONSUMER_URL}/management"
CONSUMER_PROTOCOL = f"{CONSUMER_URL}/protocol"
CONSUMER_PUBLIC = f"{CONSUMER_URL}/public"

FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}/catalog"

if (IS_LOCALHOST_DEPLOYMENT):
    PROVIDER_API = f"{PROVIDER_URL}:19191/api"
    PROVIDER_CONTROL = f"{PROVIDER_URL}:19192/control"
    PROVIDER_MANAGEMENT = f"{PROVIDER_URL}:19193/management"
    PROVIDER_PROTOCOL = f"{PROVIDER_URL}:19194/protocol"
    PROVIDER_PUBLIC = f"{PROVIDER_URL}:19291/public"

    CONSUMER_API = f"{CONSUMER_URL}:29191/api"
    CONSUMER_CONTROL = f"{CONSUMER_URL}:29192/control"
    CONSUMER_MANAGEMENT = f"{CONSUMER_URL}:29193/management"
    CONSUMER_PROTOCOL = f"{CONSUMER_URL}:29194/protocol"
    CONSUMER_PUBLIC = f"{CONSUMER_URL}:29291/public"
    
    FEDERATED_CATALOG_URL = f"{FEDERATED_CATALOG_BASE_URL}:9181/catalog"

In [46]:
"""
The vars below are useful when working on local deployment
(or at least when no routings are specified for connectors)
Docker containers have their own localhost, which is not the host
machine's localhost.

In some requests there is a 'counterPartyAddress' which contains
localhost. This one will be solved as containers' internal localhost
but not the localhost of the host machine and connectors won't connect
to each other.

Hence we substitute the "localhost" with containers' names (if they
contain "localhost", otherwise urls remain unchanged).
If routings are specified correctly on PaaS, those lines aren't
required.

We leave those vars here anyway, just not to complicate 
any of the requests later in the demo. 
"""

provider_control_internal = PROVIDER_CONTROL.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_public_internal = PROVIDER_PUBLIC.replace(LOCALHOST, PROVIDER_CONTAINER)
provider_protocol_internal = f"{PROVIDER_PROTOCOL}".replace(LOCALHOST, PROVIDER_CONTAINER)

default_headers = {
    "Content-Type": "application/json"
}

### Conn Check

In [47]:
print(f"{PROVIDER_API}/check/health/")
rp = requests.get(f"{PROVIDER_API}/check/health/").json()
print(rp)

https://provider-edc-connector.apps.dcw1.paas.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}


In [48]:
print(f"{PROVIDER_API}/check/health/")
rp = requests.get(f"{PROVIDER_API}/check/health/").json()
print(rp)

print()

print(f"{CONSUMER_API}/check/health/")
rc = requests.get(f"{CONSUMER_API}/check/health/").json()
print(rc)

print()
print("They're Alive!")

https://provider-edc-connector.apps.dcw1.paas.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}

https://consumer-edc-connector.apps.dcw1.paas.psnc.pl/api/check/health/
{'componentResults': [{'failure': None, 'component': 'Dataplane Self Registration', 'isHealthy': True}, {'failure': None, 'component': 'BaseRuntime', 'isHealthy': True}], 'isSystemHealthy': True}

They're Alive!


### Populating the provider

Not required to run, since connectors self-register data plane on startup. Just provided as an example.

In [7]:
def register_data_plane_instance_for_provider():
    global provider_control_intern
    global provider_public_intern

    return requests.post(
            headers=default_headers,
            data=json.dumps({
                "@context": {
                    "edc": "https://w3id.org/edc/v0.0.1/ns/"
                },
                "@id": "provider-dataplane",
                "url": f"{provider_control_intern}/transfer",
                "allowedSourceTypes": [ "HttpData" ],
                "allowedDestTypes": [ "HttpProxy", "HttpData" ],
                "properties": {
                        "https://w3id.org/edc/v0.0.1/ns/publicApiUrl": f"{provider_public_intern}/"
                }
            }),
            url=f"{PROVIDER_MANAGEMENT}/v2/dataplanes"
        )

register_data_plane_instance_for_provider().status_code

NameError: name 'provider_control_intern' is not defined

In [49]:
asset_id = "test-asset"

asset = create_asset(asset_id, PROVIDER_MANAGEMENT, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
asset.status_code

409

In [50]:
policy_id = "test-policy"

policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[])


"""
example with rule:

allowed_policy_region = "pl"
allowed_region_rule = {
    "action": "use", 
    "constraint": { 
        "@type": "AtomicConstraint", 
        "leftOperand": "https://w3id.org/edc/v0.0.1/ns/regionLocation", 
        "operator": "odrl:eq", 
        "rightOperand": allowed_policy_region 
    }
}
policy = create_policy(policy_id, PROVIDER_MANAGEMENT, default_headers, permissions=[allowed_region_rule])
"""

# expected 200 or 409 (already exists) because the @id is fixed in the example
print(policy.status_code)

409


Contract definition - Links an asset with a policy

In [51]:
contract_definition_id = "test-contract-definition"

contract_definition = create_contract_definition(contract_definition_id, PROVIDER_MANAGEMENT, asset_id, policy_id, default_headers)

# expected 200 or 409 (already exists) because the @id is fixed in the example
contract_definition.status_code

409

### Working on consumer side

Fetch catalog, negotiate and perform transfer

In [55]:
import time

def get_offer_id(fetched_catalog, asset_id):

    catalogs_array = []
    if isinstance(fetched_catalog, list):
        catalogs_array = fetched_catalog
    else:
        catalogs_array = [fetched_catalog]

    offer_id = None
    for catalog in catalogs_array:
        dcat_dataset = catalog["dcat:dataset"]
        dataset_array = []
        if isinstance(dcat_dataset, list):
            dataset_array = dcat_dataset
        else:
            dataset_array = [dcat_dataset]

        for asset in dataset_array:
            if (asset["@id"] == asset_id):
                policy = asset["odrl:hasPolicy"]
                if isinstance(policy, list):
                    return policy[0]["@id"]
                else:
                    return policy["@id"]
    return offer_id

# 1. fetch catalog
fetched_catalog = fetch_catalog(FEDERATED_CATALOG_URL)
print(f"1. Catalog fetch: {fetched_catalog.status_code}")
print(fetched_catalog.json())

offer_id = get_offer_id(fetched_catalog.json(), asset_id)
print(offer_id)

# 2. negotiate a contract
negotiated_contract = negotiate_contract(
    offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [], default_headers)
# If there's policy with rules then negotiated_contract = negotiate_contract(offer_id, CONSUMER_MANAGEMENT, provider_protocol_internal, [allowed_region_rule], default_headers)

print(f"2. Negotiate a contract: {negotiated_contract.status_code}")
print(negotiated_contract.json())

# wait a little until the negotiation will be finalized (automatic process which basically checks whether policies are met)
time.sleep(5)
"""
contract_negotiation_id = negotiated_contract.json()["@id"]
contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
print(contract_agreement.json())

contract_agreement_id = ""
if ("contractAgreementId" in contract_agreement.json()):
    contract_agreement_id = contract_agreement.json()["contractAgreementId"]

print(contract_agreement_id)
"""

# 3. transfer asset
"""
requested_transfer = request_consumer_pull_transfer(
    "provider",
    CONSUMER_MANAGEMENT,
    "http://consumer-backend:4000/edr-endpoint",
    provider_protocol_internal,
    contract_agreement_id,
    default_headers
)

print(f"3. Transfer an asset: {requested_transfer.status_code}")
print(requested_transfer.json())
"""

1. Catalog fetch: 200
{'@id': '6dacf841-dea3-4d02-8c8f-4ef5618eb8f9', '@type': 'dcat:Catalog', 'dcat:dataset': [{'@id': 'test-dcw1-asset', '@type': 'dcat:Dataset', 'odrl:hasPolicy': [{'@id': 'Mw==:dGVzdC1kY3cxLWFzc2V0:NDcwOTI3NjItMThhMi00NmVkLWJhOTktZTQxM2I5MGIxYTA4', '@type': 'odrl:Offer', 'odrl:permission': [], 'odrl:prohibition': [], 'odrl:obligation': []}, {'@id': 'dGVzdC1kY3cxLWNvbnRyYWN0LWRlZmluaXRpb24=:dGVzdC1kY3cxLWFzc2V0:YTVjMmMxNmEtYjQzYi00NWU1LTgzYWMtMDVlNmFjYmZlMmI1', '@type': 'odrl:Offer', 'odrl:permission': {'odrl:action': {'@id': 'odrl:use'}, 'odrl:constraint': [{'odrl:leftOperand': {'@id': 'edc:timeInterval'}, 'odrl:operator': {'@id': 'odrl:gteq'}, 'odrl:rightOperand': '2025-01-07T13:27:15.000Z'}, {'odrl:leftOperand': {'@id': 'edc:timeInterval'}, 'odrl:operator': {'@id': 'odrl:lteq'}, 'odrl:rightOperand': '2025-01-07T15:27:16.000Z'}]}, 'odrl:prohibition': [], 'odrl:obligation': []}], 'dcat:distribution': [{'@type': 'dcat:Distribution', 'dct:format': {'@id': 'HttpData-PU

'\nrequested_transfer = request_consumer_pull_transfer(\n    "provider",\n    CONSUMER_MANAGEMENT,\n    "http://consumer-backend:4000/edr-endpoint",\n    provider_protocol_internal,\n    contract_agreement_id,\n    default_headers\n)\n\nprint(f"3. Transfer an asset: {requested_transfer.status_code}")\nprint(requested_transfer.json())\n'

In [56]:
contract_negotiation_id = negotiated_contract.json()["@id"]
contract_agreement = get_contract_agreement_id(contract_negotiation_id, CONSUMER_MANAGEMENT, default_headers)
print(contract_agreement.json())

{'@type': 'ContractNegotiation', '@id': '9889bcef-7e16-46fb-9914-c95638e2f04a', 'type': 'CONSUMER', 'protocol': 'dataspace-protocol-http', 'state': 'FINALIZED', 'counterPartyId': 'provider', 'counterPartyAddress': 'https://provider-edc-connector.apps.dcw1.paas.psnc.pl/protocol', 'callbackAddresses': [], 'createdAt': 1737394792155, 'contractAgreementId': '79300af7-f820-42a7-ac2c-f9adc74571f0', '@context': {'@vocab': 'https://w3id.org/edc/v0.0.1/ns/', 'edc': 'https://w3id.org/edc/v0.0.1/ns/', 'odrl': 'http://www.w3.org/ns/odrl/2/'}}


In [19]:
contract_negotiation_id = negotiated_contract.json()["@id"]
contract_agreement = get_contract_agreement_id(contract_negotiation_id, PROVIDER_MANAGEMENT, default_headers)
print(contract_agreement.json())

[{'message': 'Object of type ContractNegotiation with ID=90f79bef-27a9-43b1-a217-af57409d3acf was not found', 'type': 'ObjectNotFound', 'path': None, 'invalidValue': None}]


In [68]:
transfer_id = requested_transfer.json()["@id"]

get_transfer_state(CONSUMER_MANAGEMENT, transfer_id)
# STARTED state means transfer was done successfully

{'@type': 'TransferState',
 'state': 'STARTED',
 '@context': {'@vocab': 'https://w3id.org/edc/v0.0.1/ns/',
  'edc': 'https://w3id.org/edc/v0.0.1/ns/',
  'odrl': 'http://www.w3.org/ns/odrl/2/'}}